In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
#hide
! pip freeze | grep fastbook || pip install -Uqq fastbook
import fastbook
fastbook.setup_book() # Only needed in colab

In [ ]:
#hide
from fastai.vision.all import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
(path/'train').ls()

In [ ]:
(path/'train/3').ls()

In [ ]:
threes = [Image.open(x) for x in (path/'train/3').ls()]
sevens = [Image.open(x) for x in (path/'train/7').ls()]
threes_v = [Image.open(x) for x in (path/'valid/3').ls()]
sevens_v = [Image.open(x) for x in (path/'valid/7').ls()]

In [ ]:
sevens_v[3]

In [ ]:
t3 = torch.stack([tensor(x) for x in threes])
t7 = torch.stack([tensor(x) for x in sevens])
y3 = tensor([0] * len(threes))
y7 = tensor([1] * len(sevens))

In [ ]:
t3_v = torch.stack([tensor(x) for x in threes_v])
t7_v = torch.stack([tensor(x) for x in sevens_v])
y3_v = tensor([0] * len(threes_v))
y7_v = tensor([1] * len(sevens_v))

In [ ]:
y = torch.cat((y3, y7))
y.sum()

In [ ]:
images = torch.cat((t3, t7), 0).view(-1, 28 * 28).float() / 255
images.shape

In [ ]:
weights = tensor([0.00001] * (28 * 28)).requires_grad_()
weights.shape

In [ ]:
torch.randn((3,1))

In [ ]:
bias = tensor(.1).requires_grad_()

In [ ]:
data_set = list(zip(images, y))

In [ ]:
images_v = torch.cat((t3_v, t7_v), 0).view(-1, 28 * 28).float() / 255
data_set_v = list(zip(images_v, torch.cat((y3_v, y7_v))))

In [ ]:
dl = DataLoader(data_set, batch_size=256, shuffle=True)
dl_v = DataLoader(data_set, batch_size=256, shuffle=True)

In [ ]:
torch.randn(20)

In [ ]:
1.0 / (1 + np.exp(-10))

In [ ]:
a = torch.Tensor(3)
a

In [ ]:
def init_params(size, std=1.0): return (torch.randn(size)*std).requires_grad_()

def validate(weights, bias, dl):
  accuracy = tensor([])

  for batch in dl:
    [image_batch, y_batch] = batch
    predictions = (image_batch @ weights + bias).sigmoid()    
    accuracy = torch.cat((accuracy, torch.where(y_batch == 1, predictions >= 0.5, predictions < 0.5).float()), 0)
  return round(accuracy.mean().item(), 4)

def train_epoch(weights, bias, dl):
  epochs = 20
  accuracy = tensor([])
  for i in range(epochs):
    for batch in dl:
      [image_batch, y_batch] = batch
      predictions = (image_batch @ weights + bias).sigmoid()
      # print('predictions', (image_batch @ weights + bias)[0:10])
      # print(image_batch[0])
      # print(image_batch @ weights + bias)
      errors = torch.where(y_batch == 1, 1 - predictions, predictions)
      
      accuracy = torch.cat((accuracy, torch.where(y_batch == 1, predictions >= 0.5, predictions < 0.5).float()), 0)
      # accuracy = torch.where(y_batch == 1, predictions >= 0.5, predictions < 0.5).float()

      # print('accuracy current', accuracy.mean())
      loss = errors.mean()
      # print('loss', loss)
      
      loss.backward()
      step = 1

      # print('bias', bias.data, 'grad', bias.grad, 'weights', weights.data.mean(), 'grad', weights.grad.mean())
      # print('weights', weights[0:10])
      weights.data -=  weights.grad.data * step
      # print('bias before', bias.data, bias.grad.data)
      bias.data -= bias.grad.data * step
      
      
      weights.grad = None
      bias.grad = None

    print(validate(weights, bias, dl_v), end = ' ')

weights = init_params((28 * 28))
bias = init_params(1)
train_epoch(weights, bias, dl)
